In [1]:
from keras.models import load_model
model = load_model('s2s.hdf5')

In [11]:
import numpy as np
import pickle

In [2]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    1227200     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    748500      ['input_2[0][0]']                
                                                                                              

In [4]:
encoder_model = load_model("encoder_model.hdf5")
decoder_model = load_model("decoder_model.hdf5")

In [5]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1227200   
                                                                 
 lstm (LSTM)                 [(None, 512),             1255424   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 2,482,624
Trainable params: 2,482,624
Non-trainable params: 0
_________________________________________________________________


In [6]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    748500      ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                            

In [23]:
with open('input_token_index.pickle', 'rb') as handle:
    input_token_index = pickle.load(handle)
    
with open('target_token_index.pickle', 'rb') as handle:
    target_token_index = pickle.load(handle)
    
# Reverse-lookup token index to decode sequences back 
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [18]:
max_encoder_seq_length = 29

def respond(text):
    input_seq = np.zeros(
        (1, max_encoder_seq_length), dtype="float32"
    )
    
    for t, word in enumerate(text.split()):
        input_seq[0, t] = input_token_index[word]
        
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['bos']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'eos' or
           len(decoded_sentence) > 50):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_char
            
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [24]:
respond("how are you")

1/1 [==============================] - 0s 19ms/step


' i do like the nba'

In [32]:
import string

eos_token = target_token_index['eos']


# basic preprocessing
def process(text):
    text = text.lower().replace('\n', ' ').replace('-', ' ').replace(':', ' ').replace(',', '') \
          .replace('"', ' ').replace(".", " ").replace("!", " ").replace("?", " ").replace(";", " ").replace(":", " ")

    text = "".join(v for v in text if v not in string.punctuation).lower()
    #text = text.encode("utf8").decode("ascii",'ignore')

    text = " ".join(text.split())
    #text+="<eos>"
    return text

In [28]:
from math import log
def generate_beam_text(seed_text, next_words, beam_search_n, break_at_eos):
    
    distributions_scores_states = [[list(), 0.0, [None, None]]]
    
    decoder_states_value = None
    
    answers = []
    
    for _ in range(next_words):
        
        sequence_temp_candidates = list()
        
        for i in range(len(distributions_scores_states)): 
            
            input_seq = np.zeros(
                (1, max_encoder_seq_length), dtype="float32"
            )
            
            # Generate empty target sequence of length 1.
            target_seq = np.zeros((1,1))
            
            seq, score, states_values = distributions_scores_states[i]
            
            if len(distributions_scores_states) == 1:
                for t, word in enumerate(process(seed_text).split()):
                    input_seq[0, t] = input_token_index[word]
                
                # Encode the input as state vectors.
                decoder_states_value = encoder_model.predict(input_seq)
                
                # Populate the first character of target sequence with the start character.
                target_seq[0, 0] = target_token_index['bos']
                
            else:
                target_seq[0, 0] = seq[-1]
                decoder_states_value = states_values
                
                candidate_sentence = ""
                for token_index in seq:
                    if token_index == eos_token:
                        break
                        
                    word = reverse_target_char_index[token_index]
                    candidate_sentence+=word + " "
                
                #print("score :", score, " | ", candidate_sentence)
                answers.append((score, candidate_sentence))
            
            
            output_tokens_distribution, h, c = decoder_model.predict([target_seq] + decoder_states_value)
            
            # Update states
            decoder_states_value = [h, c]

            predicted_distribution = output_tokens_distribution[0][0]
            
            for j in range(len(predicted_distribution)):
                if predicted_distribution[j] > 0:
                    candidate = [seq + [j], score - log(predicted_distribution[j]), decoder_states_value]
                    if break_at_eos and j == eos_token:
                        continue
                    else:
                        sequence_temp_candidates.append(candidate)

        
        # 2. score and sort all candidates
        ordered = sorted(sequence_temp_candidates, key=lambda tup:tup[1])
        
        distributions_scores_states = ordered[:beam_search_n]
          
        #print("-----")
    
    final_return_list = [x[1] for x in sorted(answers, reverse = True)[:5]]
    return(final_return_list)

In [35]:
generate_beam_text("hi do you like to dance", 5, 5, False)

1/1 [==============================] - 0s 19ms/step


['yes i do like ', 'i do like the ', 'yes i do do ', 'i do i love ', 'i do i ']